In [0]:
# from google.colab import files

In [0]:
!unzip KITTI_Data

In [0]:
import tensorflow as tf
from keras.models import Model
import keras.backend as K
from keras import layers, models, utils
import keras
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image


In [0]:
Hi,Wi= int(376/2),int(1240/2)
H,W,C = 128,512,3
batch_size=4
images = 200
training_count = 180

In [0]:
train = np.zeros((images,H,W,C,2), dtype=np.float32)
disparity = np.zeros((images,H,W), dtype=np.float32)

for i in range(200):
  if i < 10:
    train[i,:,:,:,0] = np.array(Image.open(f"KITTI_Data/Left_Images/00000{i}_10.png").resize((Wi,Hi)))[-H:,:W]
    train[i,:,:,:,1] = np.array(Image.open(f"KITTI_Data/Right_Images/00000{i}_10.png").resize((Wi,Hi)))[-H:,:W]
    disparity[i,:,:] = np.array(Image.open(f"KITTI_Data/DisparityMap/00000{i}_10.png").resize((Wi,Hi)))[-H:,:W]
  elif i < 100:
    train[i,:,:,:,0] = np.array(Image.open(f"KITTI_Data/Left_Images/0000{i}_10.png").resize((Wi,Hi)))[-H:,:W]
    train[i,:,:,:,1] = np.array(Image.open(f"KITTI_Data/Right_Images/0000{i}_10.png").resize((Wi,Hi)))[-H:,:W]
    disparity[i,:,:] = np.array(Image.open(f"KITTI_Data/DisparityMap/0000{i}_10.png").resize((Wi,Hi)))[-H:,:W]
  else:
    train[i,:,:,:,0] = np.array(Image.open(f"KITTI_Data/Left_Images/000{i}_10.png").resize((Wi,Hi)))[-H:,:W]
    train[i,:,:,:,1] = np.array(Image.open(f"KITTI_Data/Right_Images/000{i}_10.png").resize((Wi,Hi)))[-H:,:W]
    disparity[i,:,:] = np.array(Image.open(f"KITTI_Data/DisparityMap/000{i}_10.png").resize((Wi,Hi)))[-H:,:W]
disparity = disparity/200.0
train = train/255.0

In [0]:
class MultiplyDimension(layers.Layer):

  def call(self, x):
    pos = np.arange(int(x.shape[1]), dtype=np.float32)
    arr = np.zeros((int(x.shape[0]),int(x.shape[1]),int(x.shape[2]),int(x.shape[3]),int(x.shape[4])),dtype=np.float32)+pos[np.newaxis,:,np.newaxis,np.newaxis,np.newaxis]
    pos = tf.convert_to_tensor(arr)
    x *= pos
    return x

class ShiftRight(layers.Layer):
  def __init__(self, shift_count, left = True, **kwargs):
    self.shift_count = shift_count
    self.left = left
    super(ShiftRight, self).__init__(**kwargs)


  def call(self, right):
    if (self.left == True): ##Remove first few rows of left
      x = tf.concat([tf.zeros([tf.shape(right)[0],right.shape[1],self.shift_count,right.shape[3]]),right[:,:,self.shift_count:]],axis=2)
    else: ##Remove last few rows of right
      x = tf.concat([tf.zeros([tf.shape(right)[0],right.shape[1],self.shift_count,right.shape[3]]),right[:,:,:-self.shift_count]],axis=2)
    return x

def conv2(x,filter_count,kernel_size=(3,3),stride=1,dilation=(1,1),padding='same',use_bias=True,alpha=0.1,ifrelu=True,ifNorm=True,ifUpsample=False,output_padding=[0,0]):
  if (ifUpsample == True):
    x = layers.Conv2DTranspose(filters=filter_count,kernel_size=kernel_size,strides=(stride,stride),padding=padding,use_bias=use_bias,output_padding=output_padding)(x) 
  else:
    x = layers.Conv2D(filters=filter_count,kernel_size=kernel_size,strides=(stride,stride),dilation_rate=dilation,padding=padding,use_bias=use_bias)(x)
  if (ifNorm == True):
    x = layers.BatchNormalization()(x)
  if (ifrelu == True):
    layers.LeakyReLU(alpha=alpha)(x)
  return x


def conv3(x,filter_count,kernel_size=(3,3,3),stride=1,padding='same',use_bias=True,alpha=0.1,ifrelu=True,ifNorm=True,ifUpsample=False,output_padding=[0,0,0]):
  if (ifUpsample == True):
    x = layers.Conv3DTranspose(filters=filter_count,kernel_size=kernel_size,strides=(stride,stride,stride),padding=padding,use_bias=use_bias,output_padding=output_padding)(x) 
  else:
    x = layers.Conv3D(filters=filter_count,kernel_size=kernel_size,strides=(stride,stride,stride),padding=padding,use_bias=use_bias)(x)
  if (ifNorm == True):
    x = layers.BatchNormalization()(x)
  if (ifrelu == True):
    layers.LeakyReLU(alpha=alpha)(x)
  return x

def afterConv(x):
  x = layers.BatchNormalization()(x)
  x = layers.LeakyReLU(alpha=0.2)(x)
  return x

def makeBranch(x,branchx,pool_size):
  branch = layers.AveragePooling2D(pool_size=(pool_size,pool_size))(x)
  branch = branchx(branch)
  branch = afterConv(branch)
  branch = layers.Lambda( 
            lambda image: tf.image.resize_images( 
              image, (H//4, W//4), 
              method = tf.image.ResizeMethod.BILINEAR,
              align_corners = True, preserve_aspect_ratio = False
            )
          )(branch)
  return branch

def SPP(x,y,branch1,branch2,branch3,branch4,fusion1,fusion2, filter_count = int(16/4)):
  branch1 = makeBranch(x,branch1,int(filter_count*8))
  branch2 = makeBranch(x,branch2,int(filter_count*4))
  branch3 = makeBranch(x,branch3,int(filter_count*2))
  branch4 = makeBranch(x,branch4,filter_count)
  
  fusion = layers.Concatenate()([x,y,branch1,branch2,branch3,branch4])
  fusion = fusion1(fusion)
  fusion = afterConv(fusion)
  fusion = fusion2(fusion)
  fusion = afterConv(fusion)
  return fusion

def cost_volume(x,y):
  return layers.Concatenate()([x,y])


def ResLayer(x,filter_count):
  y = conv3(x,filter_count)
  y = conv3(y,filter_count)
  y = layers.add([y,x])
  return y


def CNN3D_Basic(x,skipcount,filter_count=16):
  x = conv3(x,filter_count)
  x = conv3(x,filter_count)
  for i in range(4):
    x = ResLayer(x,filter_count)

  x = conv3(x,filter_count)
  x = conv3(x,1)
  x = layers.UpSampling3D(size=(skipcount,4,4))(x)
  x = layers.Softmax(axis=1)(x)
  x = MultiplyDimension()(x)
  x = layers.Lambda(lambda xin: K.sum(xin, axis=1))(x)
  return x

def CNN3D(x,skipcount=4,base_filter_count=16):
  x = conv3(x,base_filter_count)
  x = conv3(x,base_filter_count)
  y = conv3(x,base_filter_count)
  y = conv3(y,base_filter_count)

  x = layers.add([x,y])
  y = conv3(x,base_filter_count,stride=2) ## 1/8
  y = conv3(y,base_filter_count)

  z = conv3(y,base_filter_count,stride=2) ## 1/16
  z = conv3(z,base_filter_count)
  z = conv3(z,base_filter_count,stride=2,ifUpsample=True) ## 1/8
  z = layers.add([z,y])

  a = conv3(z,base_filter_count,stride=2,ifUpsample=True,output_padding=[1,1,1]) ##1/4
  a = layers.add([a,x])

  aside_1 = conv3(a,base_filter_count)
  aside_1 = conv3(aside_1,base_filter_count)
  output1 = layers.UpSampling3D(size=(skipcount,4,4))(aside_1)
  output1 = layers.Softmax(axis=1)(output1)
  output1 = MultiplyDimension()(output1)
  output1 = layers.Lambda(lambda xin: K.sum(xin, axis=1))(output1)

  a = conv3(a,base_filter_count,stride=2) ## 1/8
  a = conv3(a,base_filter_count)
  z = layers.add([a,z])

  z = conv3(z,base_filter_count,stride=2) ## 1/16
  z = conv3(z,base_filter_count)
  z = conv3(z,base_filter_count,stride=2,ifUpsample=True) ## 1/8
  z = layers.add([z,y])

  a = conv3(z,base_filter_count,stride=2,ifUpsample=True,output_padding=[1,1,1]) ## 1/4
  a = layers.add([a,x])

  aside_2 = conv3(a,base_filter_count)
  aside_2 = conv3(aside_2,base_filter_count)
  aside_2 = layers.add([aside_2,aside_1])
  output2 = layers.UpSampling3D(size=(skipcount,4,4))(aside_2)
  output2 = layers.Softmax(axis=1)(output2)
  output2 = MultiplyDimension()(output2)
  output2 = layers.Lambda(lambda xin: K.sum(xin, axis=1))(output2)

  a = conv3(a,base_filter_count,stride=2) ## 1/8
  a = conv3(a,base_filter_count)
  z = layers.add([a,z])
  z = conv3(z,base_filter_count,stride=2) ## 1/16
  z = conv3(z,base_filter_count)
  z = conv3(z,base_filter_count,stride=2,ifUpsample=True) ## 1/8
  y = layers.add([z,y])

  y = conv3(y,base_filter_count,stride=2,ifUpsample=True,output_padding=[1,1,1]) ## 1/4
  x = layers.add([y,x])

  aside_3 = conv3(x,base_filter_count)
  aside_3 = conv3(aside_3,base_filter_count)
  aside_3 = layers.add([aside_3,aside_2])
  output3 = layers.UpSampling3D(size=(skipcount,4,4))(aside_3)
  output3 = layers.Softmax(axis=1)(output3)
  output3 = MultiplyDimension()(output3)
  output3 = layers.Lambda(lambda xin: K.sum(xin, axis=1))(output3)

  return [output1,output2,output3]

def sharedCNN(x, layers = [], arraysOfLayers = []):
  conv2_16 = x
  for i in layers:
    x = i(x)
    x = afterConv(x)
  for i in arraysOfLayers:
    for j in range(len(i)):
      x = i[j](x)
      x = afterConv(x)
      if j == 15:
        conv2_16 = x
  conv4_3 = x
  return [conv2_16,conv4_3]


def PSMN(left_input,right_input,disparity = -1,skipcount=4, base_filter_count = 16,basic3DCNN = True):
  if disparity==-1:
    disparity = right_input.shape[2]//4
  filter_count = base_filter_count; kernel_size = 3; dilation = (1,1); padding = 'same'; use_bias = True; stride = 1

  ##CNN Layers
  conv0_1 = layers.Conv2D(filters=filter_count,kernel_size=kernel_size,strides=(2,2),dilation_rate=dilation,padding=padding,use_bias=use_bias)
  conv0_2 = layers.Conv2D(filters=filter_count,kernel_size=kernel_size,strides=(stride,stride),dilation_rate=dilation,padding=padding,use_bias=use_bias)
  conv0_3 = layers.Conv2D(filters=filter_count,kernel_size=kernel_size,strides=(stride,stride),dilation_rate=dilation,padding=padding,use_bias=use_bias)
  conv1x = []
  conv2x = []
  conv3x = []
  conv4x = []

  for i in range(3):
    conv1x.append(layers.Conv2D(filters=filter_count,kernel_size=kernel_size,strides=(stride,stride),dilation_rate=dilation,padding=padding,use_bias=use_bias))

  filter_count = int(base_filter_count*2)
  conv2x.append(layers.Conv2D(filters=filter_count,kernel_size=kernel_size,strides=(2,2),dilation_rate=dilation,padding=padding,use_bias=use_bias))
  for i in range(1,16):
    conv2x.append(layers.Conv2D(filters=filter_count,kernel_size=kernel_size,strides=(stride,stride),dilation_rate=dilation,padding=padding,use_bias=use_bias))

  filter_count = int(base_filter_count*4)
  dilation = (2,2)
  for i in range(3):
    conv3x.append(layers.Conv2D(filters=filter_count,kernel_size=kernel_size,strides=(stride,stride),dilation_rate=dilation,padding=padding,use_bias=use_bias))
  
  dilation = (4,4)
  for i in range(3):
    conv4x.append(layers.Conv2D(filters=filter_count,kernel_size=kernel_size,strides=(stride,stride),dilation_rate=dilation,padding=padding,use_bias=use_bias))

  ###SPP LAYERS
  filter_count = base_filter_count
  branch_1 = layers.Conv2D(filters=filter_count,kernel_size=kernel_size,strides=(stride,stride),dilation_rate=dilation,padding=padding,use_bias=use_bias)
  branch_2 = layers.Conv2D(filters=filter_count,kernel_size=kernel_size,strides=(stride,stride),dilation_rate=dilation,padding=padding,use_bias=use_bias)
  branch_3 = layers.Conv2D(filters=filter_count,kernel_size=kernel_size,strides=(stride,stride),dilation_rate=dilation,padding=padding,use_bias=use_bias)
  branch_4 = layers.Conv2D(filters=filter_count,kernel_size=kernel_size,strides=(stride,stride),dilation_rate=dilation,padding=padding,use_bias=use_bias)
  fusion1 = layers.Conv2D(filters=int(base_filter_count*4),kernel_size=kernel_size,strides=(stride,stride),dilation_rate=dilation,padding=padding,use_bias=use_bias)
  fusion2 = layers.Conv2D(filters=base_filter_count,kernel_size=(1,1),strides=(stride,stride),dilation_rate=dilation,padding=padding,use_bias=use_bias)


  ##Make Cost Volume
  costvolume_arr = []
  left = left_input
  right = right_input
  for i in range(disparity//skipcount): 
    if i != 0:
      right = ShiftRight(skipcount,left=False)(right); left = ShiftRight(skipcount, left=True)(left)
    
    rconv4_3,rconv2_16 = sharedCNN(right,[conv0_1,conv0_2,conv0_3],[conv1x,conv2x,conv3x,conv4x])
    r_spp = SPP(rconv4_3,rconv2_16,branch_1,branch_2,branch_3,branch_4,fusion1,fusion2,filter_count=int(base_filter_count/4))
    lconv4_3,lconv2_16 = sharedCNN(left,[conv0_1,conv0_2,conv0_3],[conv1x,conv2x,conv3x,conv4x])
    l_spp = SPP(lconv4_3,lconv2_16,branch_1,branch_2,branch_3,branch_4,fusion1,fusion2,filter_count=int(base_filter_count/4))

    costvolume_i = cost_volume(l_spp,r_spp)
    costvolume_i = layers.Reshape((int(1),int(costvolume_i.shape[1]),int(costvolume_i.shape[2]),int(costvolume_i.shape[3])))(costvolume_i)
    costvolume_arr.append(costvolume_i)
  
  costvolume = layers.Concatenate(axis=1)(costvolume_arr)

  ##Model Output and Make Model
  if basic3DCNN == True:
    output = CNN3D_Basic(costvolume,skipcount,filter_count=base_filter_count)
    return Model(inputs=[left_input,right_input],outputs=[output])
  else:
    output1,output2,output3 = CNN3D(costvolume,skipcount,filter_count=base_filter_count)
    return Model(inputs=[left_input,right_input],outputs=[output1,output2,output3])





In [0]:
left_input = layers.Input(batch_shape=(batch_size,H,W,C))
right_input = layers.Input(batch_shape=(batch_size,H,W,C))

psmn = PSMN(left_input,right_input,disparity=192,skipcount=4,base_filter_count=32,basic3DCNN=False)
psmn.summary()


In [0]:
# class SaveWeights(tf.keras.callbacks.Callback):
#   def on_epoch_end(self, epoch, logs=None):
    #keras.models.save_model(psmn,'psmn.h5')
    #files.download('psmn.h5')
def smoothL1(y_true, y_pred, HUBER_DELTA = 1.0):
   x   = K.abs(y_true - y_pred)
   x   = K.switch(x < HUBER_DELTA, 0.5 * x ** 2, HUBER_DELTA * (x - 0.5 * HUBER_DELTA))
   return  K.sum(x)
psmn.compile(optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=True),loss=smoothL1)

In [0]:
###10 MINUTES PER EPOCH ON LOCAL MACHINE
psmn.fit(x=[train[:training_count,:,:,:,0],train[:training_count,:,:,:,1]],y=[disparity[:training_count,:,:,np.newaxis]],
         batch_size=batch_size,epochs=100,validation_data=([train[training_count:,:,:,:,0],train[training_count:,:,:,:,1]],
        [disparity[training_count:,:,:,np.newaxis]]),callbacks=[keras.callbacks.ModelCheckpoint('psmn_weights.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='auto', period=1)]
         )

Train on 180 samples, validate on 20 samples
Epoch 1/100
180/180 [==============================] - 2210s 12s/step - loss: 1679806.6618 - val_loss: 1235524.7937
Epoch 2/100
180/180 [==============================] - 192s 1s/step - loss: 1128159.7264 - val_loss: 1169103.4438
Epoch 3/100
180/180 [==============================] - 193s 1s/step - loss: 1114738.3778 - val_loss: 1276468.7812
Epoch 4/100
180/180 [==============================] - 194s 1s/step - loss: 1109292.4764 - val_loss: 1230715.2563
Epoch 5/100
180/180 [==============================] - 194s 1s/step - loss: 1098530.7021 - val_loss: 1312556.4812
Epoch 6/100
180/180 [==============================] - 195s 1s/step - loss: 1085534.0222 - val_loss: 1710335.4375
Epoch 7/100
180/180 [==============================] - 196s 1s/step - loss: 1070207.6222 - val_loss: 1448171.5750
Epoch 8/100
180/180 [==============================] - 196s 1s/step - loss: 1065920.2986 - val_loss: 1708442.5000
Epoch 9/100
180/180 [====================

In [0]:
test = np.zeros((12,H,W,1))
test[0:4] = psmn.predict(x=[train[training_count-18:training_count-6:3,:,:,:,0],train[training_count-6:training_count-2,:,:,:,1]])
test[4:8] = psmn.predict(x=[train[training_count-6:training_count+6:3,:,:,:,0],train[training_count-2:training_count+2,:,:,:,1]])
test[8:12] = psmn.predict(x=[train[training_count+6:training_count+18:3,:,:,:,0],train[training_count+2:training_count+6,:,:,:,1]])

real = disparity[162:198:3]*100
test = test*100

In [0]:
f, ax = plt.subplots(6,4, gridspec_kw={'wspace':0.001,'hspace':0.35},figsize=(32,8))
print("Current Number of Epochs: 150; Left Pairs from Training Set. Right Pairs From Validation Set.")
num = np.where(np.abs(real-test[:,:,:,0])<=300,1,0)
correct = np.sum(num)/(H*W*12)
print(f"Percentage Correct (Model Output Within 3px of Actual Disparity) = {correct}")
num = np.where(np.abs(real-test[:,:,:,0])<=800,1,0)
correct = np.sum(num)/(H*W*12)
print(f"Percentage Correct (Model Output Within 8px of Actual Disparity) = {correct}")
for i in range(4):
    plt.gray()
    type_ = 'Training'
    ax[0,i].imshow(test[i,:,:,0]);  ax[0,i].axis('Off'); ax[0,i].set_title(f'Output {type_}', size=16)
    ax[1,i].imshow(real[i]);  ax[1,i].axis('Off'); ax[1,i].set_title('Truth', size=16)
    if i >1:
      type_ = 'Validation'
    ax[2,i].imshow(test[i+4,:,:,0]);  ax[2,i].axis('Off'); ax[2,i].set_title(f'Output {type_}', size=16)
    ax[3,i].imshow(real[i+4]);  ax[3,i].axis('Off'); ax[3,i].set_title('Truth', size=16)
    type_ = 'Validation'
    ax[4,i].imshow(test[i+8,:,:,0]);  ax[4,i].axis('Off'); ax[4,i].set_title(f'Output {type_}', size=16)
    ax[5,i].imshow(real[i+8]);  ax[5,i].axis('Off'); ax[5,i].set_title('Truth', size=16)
plt.show()
